In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local', appName='transformationsAndActions')

22/09/29 20:51:29 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface eno2)
22/09/29 20:51:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/29 20:51:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
rdd1 = sc.parallelize([1, 2, 3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=transformationsAndActions>

In [3]:
path_file = '/home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/'
equiposOlimpicosRDD = sc.textFile(path_file+'paises.csv') \
                        .map(lambda line: line.split(','))

In [5]:
# number of lines to display
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [6]:
#it is preferable to use tuples than lists
#number of countries
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [7]:
#number of teams per country
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [8]:
#list of teams per country
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [9]:
#here use collect() because there are only 18 teams for ARG
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: 'ARG' in l)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [10]:
#count all dataset
equiposOlimpicosRDD.count()

1185

In [11]:
#count dataset by time
equiposOlimpicosRDD.countApprox(20)

1185

In [4]:
deportistaOlimpicoRDD = sc.textFile(path_file+'deportista.csv') \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path_file+'deportista2.csv') \
                        .map(lambda l: l.split(","))

In [5]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [14]:
deportistaOlimpicoRDD.count()

135572

In [15]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [16]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [6]:
deportistaOlimpicoRDD.map(lambda l: (l[-1], l[:-1])) \
    .join(equiposOlimpicosRDD.map(lambda x: (x[0], x[2]))) \
    .take(10)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN')),
 ('199', (['3610', 'An Yulong', '1', '19', '173', '70'], 'CHN')),
 ('199', (['3611', 'An Zhongxin', '2', '23', '170', '65'], 'CHN')),
 ('199', (['4639', 'Ao Changrong', '1', '25', '173', '71'], 'CHN')),
 ('199', (['4641', 'Ao Tegen', '1', '21', '181', '90'], 'CHN'))]

In [7]:
#random values
deportistaOlimpicoRDD.map(lambda l: (l[-1], l[:-1])) \
    .join(equiposOlimpicosRDD.map(lambda x: (x[0], x[2]))) \
    .takeSample(False, 10, 25)

[('273',
  (['56134', 'Niels Christian Kold Jrgensen', '1', '21', '0', '0'], 'DEN')),
 ('624', (['13109', 'Harry Arthur Bonavia', '1', '20', '0', '0'], 'MLT')),
 ('399', (['98241', 'Eberhard Radzik', '1', '25', '168', '75'], 'GER')),
 ('96', (['83781', 'Albert Muylle', '1', '0', '0', '0'], 'BEL')),
 ('1096',
  (['101476', 'Clarence Franklin Robison', '1', '25', '192', '77'], 'USA')),
 ('967', (['18615', 'Lesley Carstens', '2', '27', '0', '0'], 'RSA')),
 ('362', (['78848', 'mile Mercier', '1', '0', '0', '0'], 'FRA')),
 ('810',
  (['94563', 'Andrzej Ryszard Pitkowski', '1', '22', '169', '68'], 'POL')),
 ('1096', (['114820', 'Jerome Steinert', '1', '28', '0', '0'], 'USA')),
 ('576', (['32099', 'Assaf ElMurr', '1', '24', '0', '0'], 'LIB'))]

In [6]:
resultado = sc.textFile(path_file+'resultados.csv') \
            .map(lambda l: l.split(','))

In [7]:
resultadoGanador = resultado.filter(lambda l: 'NA' not in l[1])

In [10]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

### Personal resolution

###### #join deportistaOlimpicoRDD and equiposOlimpicosRDD

In [8]:
equipoDeportistaOlimpico = deportistaOlimpicoRDD.map(lambda l: (l[-1], l[:-1])) \
                            .join(equiposOlimpicosRDD.map(lambda x: (x[0], x[2])))

In [36]:
equipoDeportistaOlimpico.take(1)[0][1][1]

'CHN'

In [29]:
paisesMedallas = equipoDeportistaOlimpico.map(lambda l: (l[1][0][0], (l[0], l[1][0][1:], l[1][1]))) \
    .join(resultadoGanador.map(lambda x: (x[2], x[1])))

In [20]:
paisesMedallas.take(1)[0][1][1]

'Silver'

In [11]:
paisesMedallas.filter(lambda l: '65' == l[0]).take(100)

[('65',
  (('74', ['Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Bronze'))]

###### Country medal score

In [12]:
#medal values
valoresMedallas = {
    'Gold': 7,
    'Silver': 5,
    'Bronze': 4
}

In [30]:
paisesMedallas = paisesMedallas \
                .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1]]))

In [31]:
paisesMedallas.take(5)

[('CHN', 5), ('CHN', 4), ('CHN', 5), ('CHN', 5), ('CHN', 5)]

In [32]:
from operator import add

In [35]:
conclusion = paisesMedallas.reduceByKey(add).sortBy(lambda x: x[1], ascending=False)

In [38]:
conclusion.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [39]:
sc.stop()